In [1]:
import re
import os
import numpy as np
import keras
from bs4 import BeautifulSoup
import sys
from six.moves.urllib.request import urlretrieve
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

Using TensorFlow backend.


Open up the files and define the labels

In [2]:
with open('SemEval2010_task8_training/TRAIN_FILE.TXT','r') as f:
    s = f.read()
    sentences = s.splitlines()[0::4]
    x_train =[sentence.split('\t')[1].strip('"') for sentence in sentences]
    y_train = s.splitlines()[1::4]

with open('SemEval2010_task8_testing_keys/TEST_FILE_CLEAN.TXT','r') as f:
    s = f.read()
    x_test = []
    for line in s.splitlines():
        d = line.split('\t')[1].strip('"')
        x_test.append(d)

with open('SemEval2010_task8_testing_keys/TEST_FILE_KEY.TXT','r') as f:
    s = f.read()
    y_test = [k.split('\t')[1] for k in s.splitlines()]

label_index = {'Cause-Effect': 0,
 'Component-Whole': 1,
 'Content-Container': 2,
 'Entity-Destination': 3,
 'Entity-Origin': 4,
 'Instrument-Agency': 5,
 'Member-Collection': 6,
 'Message-Topic': 7,
 'Other': 8,
 'Product-Producer': 9}

MAX_LEN = 85

In [3]:
def preprocess_labels(target, label_index=label_index, one_hot=True):
    reduced_labels = [t.replace('(e2,e1)','') for t in target]
    reduced_labels = [t.replace('(e1,e2)','') for t in reduced_labels]
    labels = [label_index[i] for i in reduced_labels]
    return labels

In [4]:
y_train = preprocess_labels(y_train)

In [5]:
train_data = {v:[x_train[x] for x in range(8000) if y_train[x] == v] for v in label_index.values()}

### Augment data to represent all classes equally
We are doubling the size of the dataset, from 8000 to 16000 by adding random duplicates of the classes that make a lesser representation

In [6]:
distribution = np.unique(y_train,return_counts=1,)[-1]/len(y_train)
inverse = (1/distribution)
inverse = inverse/inverse.sum()


for i in range(8000):
    random_class = np.random.choice(10, p=inverse)
    
    sample_class = train_data[random_class]
    sample = sample_class[np.random.choice(len(sample_class))]
    
    x_train.append(sample)
    y_train.append(random_class)

In [7]:
np.unique(y_train,return_counts=1)[-1]/16000

array([ 0.1006875,  0.096375 ,  0.0985625,  0.1010625,  0.09275  ,
        0.1059375,  0.1003125,  0.0974375,  0.1143125,  0.0925625])

### Preprocessing
We need to convert the data such that it makes more sense for a machine learning algorithm.

## Step 1 : Extract the e1 and e2 words

![](http://i.imgur.com/pmSubOi.png)

**Input :** `They saw that the <e1>equipment</e1> was put inside rollout <e2>drawers</e2>`.

**Output :** 

`They saw that the equipment was put inside rollout drawers.
[equipment, drawers]`

The sentence with the markup `<e1>.....</e1>` and `<e2>.....</e2>` are first converted to normal text. The e1 and e2 words are stored for each data sample in a separate array.



In [8]:
tokenizer = Tokenizer()
def clean_text(sentence_list): 
    soups = [BeautifulSoup(sen,"html5lib") for sen in sentence_list]
    clean_text = [soup.text for soup in soups]
    e1_words = [soup.find('e1').text.lower() for soup in soups]
    e2_words = [soup.find('e2').text.lower() for soup in soups]
    clean_text = [' '.join(text_to_word_sequence(clean)) for clean in clean_text]
    return [clean_text,e1_words,e2_words]

## Step 2: Remove all stopwords and punctuation
**Input :**  
`They saw that the equipment was put inside rollout drawers. 
[equipment, drawers]`

**Output :**  
`They saw that the equipment was put inside rollout drawers 
[equipment, drawers]`

## Step 3: Tokenize the words

**Input :**  They saw that the equipment was put inside rollout drawers. [equipment, drawers]

**Output :** 

`[23, 54, 65, 1, 1022, 55, 66, 65, 1156, 502] 
[1022, 502]`

All the words are given an index and are converted to numbers for easier processing

## Step 4: Make a one-hot-encoding of e1 and e2 words (Almost one-hot. There are some e1 and e2 phrases

![](http://i.imgur.com/TjTnAnT.png)

**Input :** 

`[23, 54, 65, 1, 1022, 55, 66, 65, 1156, 502] 
[1022, 502]`

**Output :** 

`[23, 54, 65, 1, 1022, 55, 66, 65, 1156, 502] 
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, , 0, 0, 0, 0, 1]` 

In [9]:
def preprocess(params, tokenizer = tokenizer):  
    [clean_train, e1, e2] = params
    token_sentences = tokenizer.texts_to_sequences(clean_train)
    token_e1 = tokenizer.texts_to_sequences(e1)
    token_e2 = tokenizer.texts_to_sequences(e2)
    e1_one_hots = []
    for sentence,e1 in zip(token_sentences,token_e1):
        s1 = []
        for word in sentence:
            if word in e1:
                s1.append(1)
            else:
                s1.append(0)
        e1_one_hots.append(s1)

    e2_one_hots = []
    for sentence,e2 in zip(token_sentences,token_e2):
        s1 = []
        for word in sentence:
            if word in e2:
                s1.append(1)
            else:
                s1.append(0)
        e2_one_hots.append(s1)
    padded_sentences = pad(token_sentences)
    padded_e1 = pad(token_e1)
    padded_e2 = pad(token_e2)
    return [padded_sentences, padded_e1, padded_e2]

## Step 5: Pad all sequences to equal lengths (85 words)

**Input : **

`[23, 54, 65, 1, 1022, 55, 66, 65, 1156, 502] 
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, , 0, 0, 0, 0, 1]`

**Output :**

`[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 54, 65, 1, 1022, 55, 66, 65, 1156, 502]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]`

In [10]:
def pad(sequence):
    return keras.preprocessing.sequence.pad_sequences(sequence,maxlen=MAX_LEN)

## Step 6: Convert labels into one-hot-encoded vectors

**Input: **
`Cause-Effect`

**Output: **
`[1,0,0,0,0,0,0,0,0,0]
`

In [11]:
def onehot(lab):
    labels = keras.utils.to_categorical(lab)
    return labels

Let's run all this now. This might take some time depending on the performance of your system.

In [12]:
x_train = clean_text(x_train)
tokenizer.fit_on_texts(x_train[0])

x_train = preprocess(x_train)
y_train = onehot(y_train)

x_test = preprocess(clean_text(x_test))
y_test = onehot(preprocess_labels(y_test))

print("Train Data:")
print("Sentences: {}".format(x_train[0].shape))
print("Labels: {}".format(y_train.shape))
print()
print("Test Data:")
print("Sentences: {}".format(x_test[0].shape))
print("Labels: {}".format(y_test.shape))

Train Data:
Sentences: (16000, 85)
Labels: (16000, 10)

Test Data:
Sentences: (2717, 85)
Labels: (2717, 10)


### Word Embedding
Word embeddings convert words into vectors that give it semantic meaning. These vectors are dense representation of the words and make more sense to a machine learning program. [Here](https://www.tensorflow.org/tutorials/word2vec) is a great article to get started. 

**Word Embeddings are essential, for our model to perform well**


Let's get the GLOVE word embeddings Matrix as a txt file. Download it [here](https://drive.google.com/uc?export=download&id=0B30g1WfHiiY-MmN2dVVkdnV1S2M). Place it in the same directory as this notebook.

In [13]:
link = "https://drive.google.com/uc?export=download&id=0B30g1WfHiiY-MmN2dVVkdnV1S2M"
file_name = "glove.6B.100d.txt"

if not file_name in os.listdir('.'):
    print("File not found in the directory.\nPlease download %s" % link)
    print("Make sure to place it in the same directory as this notebook")
else:
    print("File Found. Loading word2vec matrix...")

embeddings_index = {}
f = open('glove.6B.100d.txt','rb')
for line in f:
    values = line.split()
    word = values[0].decode('utf-8')
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 100))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
print("Done.")

File Found. Loading word2vec matrix...
Done.


## Merging the e1 and e1 positional vectors with the main sentence

In all of the models below, the e1 and e2 positional vectors are concatenated with the main sentence vector.

![](http://i.imgur.com/culDn1e.png)

### Model attempt 1 : Random Forest classifier

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

In [15]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = 100

    def fit(self, X, y):
        return self

    def transform(self, X):
        word_vec = np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X[0]
        ])
        
        merged = np.concatenate([word_vec, X[1], X[2]], axis=1)
        return merged

In [16]:
etree_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(embeddings_index)),
    ("extra trees", ExtraTreesClassifier(n_estimators=500))])

In [17]:
random_forest = Pipeline([
    ("word2vec", MeanEmbeddingVectorizer(embeddings_index)),
    ("random forest", RandomForestClassifier(n_estimators=100))
])

In [18]:
random_forest.fit(x_train,y_train.argmax(axis=1))

Pipeline(memory=None,
     steps=[('word2vec', <__main__.MeanEmbeddingVectorizer object at 0x000002242FB958D0>), ('random forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurit...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [19]:
(random_forest.predict(x_test) == np.argmax(y_test,axis=1)).mean()

0.26021347073978651

That's pretty bad

### Model attempt 2: SVM

In [20]:
from sklearn import svm

In [21]:
c = svm.SVC(decision_function_shape='ovo')

In [22]:
svc_classifier =  Pipeline([
    ("word2vec", MeanEmbeddingVectorizer(embeddings_index)),
    ("random forest", c)
])

In [23]:
svc_classifier.fit(x_train,y_train.argmax(axis=1))

Pipeline(memory=None,
     steps=[('word2vec', <__main__.MeanEmbeddingVectorizer object at 0x000002242FEA6E10>), ('random forest', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [24]:
(svc_classifier.predict(x_test) == np.argmax(y_test,axis=1)).mean()

0.21494295178505704

Almost as bad.

### Model attempt 3 : Deep LSTM RNN Model

## Layer 1: Input
The first layer of our network will take one input : **The main sentence**

In [25]:
sentence_input = keras.layers.Input(shape=(MAX_LEN,), name="sentence_input")

## Layer 2: Word Embeddings
The sentence input will be converted to a vector based on the word2vec matrix that we obtained earlier

In [26]:
embed_layer = keras.layers.Embedding(len(word_index)+1,100,weights=[embedding_matrix],input_length=MAX_LEN, name="word2vec")
vector_sentence = embed_layer(sentence_input)

##  Auxillary Inputs

We will feed the e1 and e2 positional vectors to the word embeddings vectors

In [27]:
e1 = keras.layers.Input(shape=(MAX_LEN,), name="e1_input")
e1r = keras.layers.Reshape((MAX_LEN,1), name="e1_reshape")(e1)
e2 = keras.layers.Input(shape=(MAX_LEN,), name="e2_input")
e2r = keras.layers.Reshape((MAX_LEN,1), name="e2_reshape")(e2)

## Layer 3: Merge Layer
This concatanates the values from the the two e1 and e2 vectors as auxillary input with the word embedding vectors.

`merged layer = e1 +e2 + word embedding`

In [28]:
merged = keras.layers.concatenate([vector_sentence,e1r,e2r], name="merge")

## Layer 4: Dropout
To prevent overfitting, half of the incoming inputs will be sqashed to 0. Yeah. It's savage and it's true.

In [29]:
x = keras.layers.Dropout(0.5)(merged)

## Layer 5: LSTM Neurons 1
We added 128 LSTM neurons that returns requences into the next LSTM array

In [30]:
x = keras.layers.LSTM(128,return_sequences=True,name="LSTM_1")(x)

## Layer 6: Dropout
Oh but before that. Dropout.

In [31]:
x = keras.layers.Dropout(0.5)(x)

## Layer 7: LSTM Neurons 2
Another 32 LSTM neurons that recieve from the first LSTM layer

In [32]:
x = keras.layers.LSTM(32)(x)

## Layer 8: Dropout
Of course

In [33]:
x = keras.layers.Dropout(0.5)(x)

## Layer 9: Softmax
The final layer that outputs our prediction is a softmax layer with 10 neurons. One for each category.

In [34]:
prediction = keras.layers.Dense(10, activation='softmax', name="prediction")(x)

## Compile
Our model is ready. Let's compile it.

In [35]:
model = keras.models.Model([sentence_input,e1,e2],prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
sentence_input (InputLayer)      (None, 85)            0                                            
____________________________________________________________________________________________________
e1_input (InputLayer)            (None, 85)            0                                            
____________________________________________________________________________________________________
e2_input (InputLayer)            (None, 85)            0                                            
____________________________________________________________________________________________________
word2vec (Embedding)             (None, 85, 100)       1960600     sentence_input[0][0]             
___________________________________________________________________________________________

We have 2,099,810 trainable parameters. That's a lot of room for over-fitting.

## Let's train
We can start the training process. All the datasamples from the train folder are being used. 

This will take a long time. About 30 mins on average.

In [36]:
# Remove validation data
model.fit(x_train,y_train, epochs=12)

Epoch 1/12
16000/16000 [==============================] - 160s - loss: 1.8488 - acc: 0.3730   
Epoch 2/12
16000/16000 [==============================] - 161s - loss: 1.4462 - acc: 0.5405   
Epoch 3/12
16000/16000 [==============================] - 159s - loss: 1.2636 - acc: 0.6031   
Epoch 4/12
16000/16000 [==============================] - 163s - loss: 1.1340 - acc: 0.6504   
Epoch 5/12
16000/16000 [==============================] - 163s - loss: 1.0323 - acc: 0.6851   
Epoch 6/12
16000/16000 [==============================] - 166s - loss: 0.9406 - acc: 0.7137   
Epoch 7/12
16000/16000 [==============================] - 165s - loss: 0.8696 - acc: 0.7321   
Epoch 8/12
16000/16000 [==============================] - 165s - loss: 0.7972 - acc: 0.7554   
Epoch 9/12
16000/16000 [==============================] - 165s - loss: 0.7419 - acc: 0.7730   
Epoch 10/12
16000/16000 [==============================] - 165s - loss: 0.6854 - acc: 0.7904   
Epoch 11/12
16000/16000 [========================

In [38]:
model.evaluate(x_test,y_test)[1]

2717/2717 [==============================] - 6s     


0.67206477721824687

Compared to the other models, this one performs the best.

Let's print out the predicted results for the first 5 test cases

In [42]:
def format_predict(array):
    reverse_label = {v:k for k,v in label_index.items()}
    return reverse_label[np.argmax(prediction)]

a = 8000
for prediction in model.predict(x_test)[:5]:
    a+=1
    print(a,format_predict(prediction))

8001 Message-Topic
8002 Product-Producer
8003 Instrument-Agency
8004 Entity-Destination
8005 Cause-Effect


It's pretty good at predicting

Let's write all this to `result.txt` 

In [47]:
results = []
a = 8000
for prediction in model.predict(x_test):
    a+=1
    results.append(str(a))
    results.append('\t')
    results.append(format_predict(prediction))
    results.append('\n')

In [49]:
result = ''.join(results)

In [51]:
with open('results.txt','w') as f:
    f.write(result)

# Improvements
* More data
* Maybe reduction of the model complexity
* Using other dimension word embeddings, especially the 50d one

# References
1. http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
2. https://keras.io/getting-started/functional-api-guide/
3. https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html